In [1]:
%load_ext autoreload
%autoreload 2

In [87]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import sklearn.model_selection as sc_model_selection
import sklearn.preprocessing as sc_preprep

In [252]:
data = pd.read_csv("data/train.csv")

# Data inspection

In [253]:
data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

# Preprocessing

In [245]:
## TODO: As Pipeline
# Impute seemingly metric vars
data["Age"] = data["Age"].fillna(data["Age"].median())
data["RoomService"] = data["RoomService"].fillna(data["RoomService"].median()) #TODO: Use mode without NA
data["FoodCourt"] = data["FoodCourt"].fillna(data["FoodCourt"].median()) #TODO: Use mode without NA
data["ShoppingMall"] = data["ShoppingMall"].fillna(data["ShoppingMall"].median()) #TODO: Use mode without NA
data["Spa"] = data["Spa"].fillna(data["Spa"].median()) #TODO: Use mode without NA
data["VRDeck"] = data["VRDeck"].fillna(data["VRDeck"].median()) #TODO: Use mode without NA
# One hot encode HomePlanet
data = pd.get_dummies(data, prefix=['HomePlanet'], columns = ['HomePlanet'], drop_first=False)
# One hot encode VIP
data = pd.get_dummies(data, prefix=['VIP'], columns = ['VIP'], drop_first=False)
# One hot encode Destination
data = pd.get_dummies(data, prefix=['Destination'], columns = ['Destination'], drop_first=False)
## One hot encode Cabin
data[["Deck", "Num", "Side"]] = data["Cabin"].str.split('/', 3, expand=True)
data["Num"] = data["Num"].fillna(data["Num"].median()) #TODO: Use mode without NA
data = pd.get_dummies(data, prefix=['Deck'], columns = ['Deck'], drop_first=False)
data = pd.get_dummies(data, prefix=['Side'], columns = ['Side'], drop_first=False)
# One hot encode CryoSleep
data = pd.get_dummies(data, prefix=['CryoSleep'], columns = ['CryoSleep'], drop_first=False)

/tmp/ipykernel_17651/922472410.py:15: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  data[["Deck", "Num", "Side"]] = data["Cabin"].str.split('/', 3, expand=True)


# Training

In [246]:
X = data.loc[:, data.columns != 'Transported']
Y = data['Transported']
X_train, X_test = sc_model_selection.train_test_split(X)
Y_train, Y_test = sc_model_selection.train_test_split(Y)

# Definitions

In [247]:
data.isnull().sum()

PassengerId                    0
Cabin                        199
Age                            0
RoomService                    0
FoodCourt                      0
ShoppingMall                   0
Spa                            0
VRDeck                         0
Name                         200
Transported                    0
HomePlanet_Earth               0
HomePlanet_Europa              0
HomePlanet_Mars                0
VIP_False                      0
VIP_True                       0
Destination_55 Cancri e        0
Destination_PSO J318.5-22      0
Destination_TRAPPIST-1e        0
Num                            0
Deck_A                         0
Deck_B                         0
Deck_C                         0
Deck_D                         0
Deck_E                         0
Deck_F                         0
Deck_G                         0
Deck_T                         0
Side_P                         0
Side_S                         0
CryoSleep_False                0
CryoSleep_

In [248]:
data.columns.values[0:1000]

array(['PassengerId', 'Cabin', 'Age', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported',
       'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'VIP_False', 'VIP_True', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'Num',
       'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F',
       'Deck_G', 'Deck_T', 'Side_P', 'Side_S', 'CryoSleep_False',
       'CryoSleep_True'], dtype=object)

In [249]:
COLUMNS = [
    ## Metric
    "Age",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
    "Num",
    ## HomePlanet
    "HomePlanet_Earth",
    "HomePlanet_Europa",
    "HomePlanet_Mars",
    ## VIP
    "VIP_True",
    "VIP_False",
    ## Dest
    "Destination_55 Cancri e",
    "Destination_PSO J318.5-22",
    "Destination_TRAPPIST-1e",
    ## Cryo
    "CryoSleep_False",
    "CryoSleep_True",
    ## Deck
    "Deck_A",
    "Deck_B",
    "Deck_C",
    "Deck_D",
    "Deck_E",
    "Deck_F",
    "Deck_G",
    "Deck_T",
    ## Side
    "Side_P",
    "Side_S",
]


## Models

In [250]:
from models.baseline_regression import get_baseline_regression_model
model = get_baseline_regression_model(X_train, Y_train, COLUMNS)

# Evaluation

In [251]:
from sklearn.metrics import accuracy_score
prediction = model.predict(X_test[COLUMNS])
accuracy_score(prediction, Y_test)

0.4871205151793928

# Prediction

In [254]:
data_predict = pd.read_csv("data/test.csv")
# Impute age
data_predict["Age"] = data_predict["Age"].fillna(data_predict["Age"].median())
# One Hot Encode Home-Planet
data_predict = pd.get_dummies(data_predict, prefix=['HomePlanet'], columns = ['HomePlanet'], drop_first=False)
# Get features
predictor = data_predict[COLUMNS]
# Predict
PassengerId = data_predict["PassengerId"]
Prediction = model.predict(predictor)

KeyError: "['Num', 'VIP_True', 'VIP_False', 'Destination_55 Cancri e', 'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'CryoSleep_False', 'CryoSleep_True', 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_T', 'Side_P', 'Side_S'] not in index"

In [109]:
pd.DataFrame(Prediction, index=PassengerId, columns=["Transported"]).to_csv("my_submission.csv")